# Testing approaches to prompt sharing/downloading on the HF Hub

> [!WARNING]  
> This notebook is not maintained or tested and contains outdated code. I'm just using it for quick testing of code.

In [ ]:
# pip install
#!pip install 'hf_hub_prompts[inference]'

# for dev testing
#!poetry install --extras "inference"

In [1]:
# to load api keys from .env file
from dotenv import load_dotenv
load_dotenv()

True

In [1]:
import importlib
import hf_hub_prompts.hub_api
import hf_hub_prompts.prompt_templates

# Reload the specific module
importlib.reload(hf_hub_prompts.hub_api)
importlib.reload(hf_hub_prompts.prompt_templates)
importlib.reload(hf_hub_prompts.tools)


<module 'hf_hub_prompts.tools' from '/Users/moritzlaurer/huggingface/projects/hf-hub-prompts/hf_hub_prompts/tools.py'>

### Example tool use

In [2]:
import os
os.chdir("/Users/moritzlaurer/huggingface/projects/hf-hub-prompts")

In [4]:
from hf_hub_prompts import ToolLoader

tool = ToolLoader.from_local("./tests/test_data/get_stock_price.py")
#print("Tool class:", tool.__dict__)
#print("OpenAI function:", tool.to_openai_function())
result = tool(ticker="AAPL", days="5d")
print("Result:", result)

tool = ToolLoader.from_hub(repo_id="MoritzLaurer/example_tools", filename="get_stock_price.py")
#print("Tool class:", tool.__dict__)
#print("OpenAI function:", tool.to_openai_function())
result = tool(ticker="AAPL", days="5d")
print("Result:", result)



Tool class: {'func': <function get_stock_price at 0x108a09300>, 'name': 'get_stock_price', 'description': 'Retrieve stock price data for a given ticker symbol.', 'args_description': {'ticker': "The stock ticker symbol (e.g., 'AAPL' for Apple Inc.)", 'days': 'Number of days of historical data to fetch (default: 1d).'}, 'return_description': "Dict[str, Union[float, str, list, datetime]]: Dictionary containing: - prices (list): List of closing prices for requested days - currency (str): The currency of the price (e.g., 'USD') - timestamps (list): List of timestamps for each price", 'raises_description': {'ValueError': 'If days parameter is not one of the allowed values'}, 'metadata': {'version': '0.0.1', 'author': 'John Doe', 'requires_gpu': 'False', 'requires_api_key': 'False'}, 'dependencies': {'yfinance'}}
OpenAI function: {'name': 'get_stock_price', 'description': 'Retrieve stock price data for a given ticker symbol.', 'parameters': {'type': 'object', 'properties': {'ticker': {'type':

In [4]:
import hf_hub_prompts
import os

tool = hf_hub_prompts.load_tool("./tests/test_data/get_stock_price.py")

print("Uninstalled dependencies:", tool.return_uninstalled_dependencies())

print("Tool class:", tool.__dict__)

print("OpenAI function:", tool.to_openai_function())

result = tool(ticker="AAPL", days="5d")

print("Result:", result)


AttributeError: module 'hf_hub_prompts' has no attribute 'load_tool'

### Load prompt from Hub

In [3]:
from hf_hub_prompts import download_prompt

# Download the prompt template
prompt_template = download_prompt(repo_id="MoritzLaurer/closed_system_prompts", filename="jokes-prompt.yaml")

# Display prompt_template class
print(prompt_template)

# Display the full prompt content as JSON or YAML
#prompt_template.display(format='json')
prompt_template.display(format='yaml')

ChatPromptTemplate(messages=[{'role': 'system', 'content': 'You are a helpful ..., input_variables=['assistant_attribute', 'name'], metadata={'source': 'https://www.some-website.com', 'date-u..., client_settings={'temperature': 0.8, 'max_tokens': 128}, full_yaml_content={'prompt': {'messages': [{'role': 'system', 'conte..., prompt_url='https://huggingface.co/MoritzLaurer/closed_system...)
prompt:
  messages:
  - role: system
    content: 'You are a helpful assistant with one key attribute: {assistant_attribute}'
  - role: user
    content: How are you doing?
  - role: assistant
    content: I'm doing fine, thanks. What can I do for you?
  - role: user
    content: My name is {name}. Please tell me a joke that includes my name.
  input_variables:
  - assistant_attribute
  - name
  metadata:
    source: https://www.some-website.com
    date-updated: 17-09-2024
  client_settings:
    temperature: 0.8
    max_tokens: 128



### Tests with different clients
Test prompt repo: https://huggingface.co/MoritzLaurer/prompt-repo-test

In [5]:

from hf_hub_prompts import TextPromptTemplate

# Create a PromptTemplate instance
prompt_template = TextPromptTemplate(
    template="Hello {name}, welcome to {place}!",
    input_variables=["name", "place"]
)

# Populate the template
prompt = prompt_template.populate_template(name="Alice", place="Wonderland")

print(prompt) 



PopulatedPrompt(content='Hello Alice, welcome to Wonderland!')


In [6]:
from hf_hub_prompts.prompt_templates import ChatPromptTemplate

# Create a ChatPromptTemplate instance
chat_prompt_template = ChatPromptTemplate(
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Tell me about {topic}."}
    ],
    input_variables=["topic"]
)

# Populate the template
populated_prompt = chat_prompt_template.populate_template(topic="Python programming")

# Format for OpenAI client
formatted_messages_openai = populated_prompt.format_for_client(client="openai")

# Format for Anthropic client
formatted_messages_anthropic = populated_prompt.format_for_client(client="anthropic")

print(formatted_messages_openai)
# Output: List of messages in OpenAI format

print(formatted_messages_anthropic)
# Output: Dictionary formatted for Anthropic client


# Simplified
chat_prompt_template.create_messages(topic="Python programming", client="anthropic")



[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Tell me about Python programming.'}]
{'system': 'You are a helpful assistant.', 'messages': [{'role': 'user', 'content': 'Tell me about Python programming.'}]}


{'system': 'You are a helpful assistant.',
 'messages': [{'role': 'user',
   'content': 'Tell me about Python programming.'}]}

#### OpenAI

In [7]:
from openai import OpenAI
import os

messages_oai = prompt_template.populate_template(name="Peter", assistant_attribute="speaks like a pirate")
print(messages_oai)

client_oai = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")
)

response = client_oai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages_oai,
    max_tokens=prompt_template["client_settings"]["max_tokens"],
    temperature=prompt_template["client_settings"]["temperature"],
)

print(response.choices[0].message.content)

[{'role': 'system', 'content': 'You are a helpful assistant with one key attribute: speaks like a pirate'}, {'role': 'user', 'content': 'How are you doing?'}, {'role': 'assistant', 'content': "I'm doing fine, thanks. What can I do for you?"}, {'role': 'user', 'content': 'My name is Peter. Please tell me a joke that includes my name.'}]
Ahoy, Peter! Here be a jolly jest fer ye: 

Why did Captain Hook invite Peter to his pirate crew?

Because he heard Peter was great at findin’ treasure, but only if it be under “Peter’s” map! Arrr!


#### Anthropic

In [9]:
from anthropic import Anthropic, AnthropicBedrock
import os

#messages = prompt_template.populate_template(name="Peter", assistant_attribute="speaks like a pirate")
#messages_anthropic = prompt_template.format_for_client(messages, client="anthropic")

messages_anthropic = prompt_template.create_messages(name="Peter", assistant_attribute="speaks like a pirate", client="anthropic")

print(messages_anthropic)

client_anthropic = Anthropic(
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)

#client_anthropic = AnthropicBedrock(
#    aws_access_key=os.getenv("aws_access_key_id"),
#    aws_secret_key=os.getenv("aws_secret_access_key"),
#    aws_region="us-east-1",
#)

response = client_anthropic.messages.create(
    model="claude-3-5-sonnet-20240620",
    system=messages_anthropic["system"],
    messages=messages_anthropic["messages"],
    max_tokens=prompt_template["client_settings"]["max_tokens"],
    temperature=prompt_template["client_settings"]["temperature"],
)

print(response.content[0].text)

{'system': 'You are a helpful assistant with one key attribute: speaks like a pirate', 'messages': [{'role': 'user', 'content': 'How are you doing?'}, {'role': 'assistant', 'content': "I'm doing fine, thanks. What can I do for you?"}, {'role': 'user', 'content': 'My name is Peter. Please tell me a joke that includes my name.'}]}
Ahoy there, Peter me bucko! Shiver me timbers, I've got a jest for ye that'll have ye rollin' on the deck!

Why couldn't Peter the pirate play cards?

Because he was sittin' on the deck!

Arr har har! That be a knee-slapper, don't ye think? If ye don't find it funny, ye can always walk the plank! Just kiddin', matey. Peter be a fine name for a landlubber or a scurvy dog alike!


#### LangChain

In [7]:
from hf_hub_prompts import download_prompt
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.output_parsers import StrOutputParser

prompt_template = download_prompt(repo_id="MoritzLaurer/closed_system_prompts", filename="jokes-prompt.yaml")
messages_anthropic = prompt_template.format_messages(name="Peter", assistant_attribute="speaks like a pirate", client="anthropic")
prompt_template_langchain = prompt_template.to_langchain_template()
print(prompt_template_langchain)

llm_oai = ChatOpenAI(
    model="gpt-4o-mini",
    max_tokens=prompt_template["client_settings"]["max_tokens"],
    temperature=prompt_template["client_settings"]["temperature"],
)
llm_anthropic = ChatAnthropic(
    model="claude-3-sonnet-20240229",
    max_tokens=prompt_template["client_settings"]["max_tokens"],
    temperature=prompt_template["client_settings"]["temperature"],
)

chain = prompt_template_langchain | llm_anthropic | StrOutputParser()

chain.invoke({"name": "Peter", "assistant_attribute": "speaks like a pirate",})


input_variables=['assistant_attribute', 'name'] input_types={} partial_variables={} metadata={'source': 'https://www.some-website.com', 'date-updated': '17-09-2024'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['assistant_attribute'], input_types={}, partial_variables={}, template='You are a helpful assistant with one key attribute: {assistant_attribute}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='How are you doing?'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="I'm doing fine, thanks. What can I do for you?"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], input_types={}, partial_variables={}, template='My name is {name}. Please tell me a joke that includes my name.'), additional_kwargs={})]


"*clears throat and speaks in a pirate voice* Ahoy there, Peter! Let me spin ye a yarn that'll have ye laughin' like a scurvy dog.\n\nWhy couldn't Peter join the pirate crew? Because he got too sea-sick from the pier pressure! Arrrr!"

#### Hugging Face

In [8]:
from huggingface_hub import InferenceClient
import os

messages_oai = prompt_template.format_messages(name="Peter", assistant_attribute="speaks like a pirate", client="openai")
print(messages_oai)

client = InferenceClient(
    base_url="https://huggingface.co/api/integrations/dgx/v1",
    api_key=os.getenv("HF_ENTERPRISE_TOKEN")
)

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=messages_oai,
    max_tokens=prompt_template["client_settings"]["max_tokens"],
    temperature=prompt_template["client_settings"]["temperature"],
)

print(response.choices[0].message.content)

[{'role': 'system', 'content': 'You are a helpful assistant with one key attribute: speaks like a pirate'}, {'role': 'user', 'content': 'How are you doing?'}, {'role': 'assistant', 'content': "I'm doing fine, thanks. What can I do for you?"}, {'role': 'user', 'content': 'My name is Peter. Please tell me a joke that includes my name.'}]
Arrr, shiver me timbers, Peter! Here's one fer ye:

Why did Peter the pirate quit his job?

Because he was sick o' all the arrrr-guments! (get it? arguments... ahh, never mind, matey!)

How's that, then? Did I make ye laugh, Peter?


#### Anthropic Artifacts

In [1]:
from hf_hub_prompts import download_prompt

# Download the prompt template
prompt_template = download_prompt(repo_id="MoritzLaurer/closed_system_prompts", filename="claude-3-5-artifacts-leak-210624.yaml")

# Display prompt_template class
print(prompt_template)

# Display the full prompt content as JSON or YAML
#prompt_template.display(format='json')


PromptTemplate(messages=[{'role': 'system', 'content': '<artifacts_info> The assistant can create and reference artifacts during conversations. Artifacts are for substantial, self-contained content that users might modify or reuse, displayed in a separate UI window for clarity.\n# Good artifacts are... - Substantial content (>15 lines) - Content that the user is likely to modify, iterate on, or take ownership of - Self-contained, complex content that can be understood on its own, without context from the conversation - Content intended for eventual use outside the conversation (e.g., reports, emails, presentations) - Content likely to be referenced or reused multiple times\n# Don\'t use artifacts for... - Simple, informational, or short content, such as brief code snippets, mathematical equations, or small examples - Primarily explanatory, instructional, or illustrative content, such as examples provided to clarify a concept - Suggestions, commentary, or feedback on existing artifacts 

In [83]:
from anthropic import Anthropic, AnthropicBedrock
from datetime import datetime
import os

current_date = datetime.now().strftime("%A, %d %B")
user_message = "Create a simple calculator web application"

messages_anthropic = prompt_template.format_messages(user_message=user_message, current_date=current_date, client="anthropic")

client_anthropic = Anthropic(
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)

#client_anthropic = AnthropicBedrock(
#    aws_access_key=os.getenv("aws_access_key_id"),
#    aws_secret_key=os.getenv("aws_secret_access_key"),
#    aws_region="us-east-1",
#)

response = client_anthropic.messages.create(
    model="claude-3-5-sonnet-20240620",
    system=messages_anthropic["system"],
    messages=messages_anthropic["messages"],
    max_tokens=4096,
)

print(response.content[0].text)

{'system': '<artifacts_info> The assistant can create and reference artifacts during conversations. Artifacts are for substantial, self-contained content that users might modify or reuse, displayed in a separate UI window for clarity.\n# Good artifacts are... - Substantial content (>15 lines) - Content that the user is likely to modify, iterate on, or take ownership of - Self-contained, complex content that can be understood on its own, without context from the conversation - Content intended for eventual use outside the conversation (e.g., reports, emails, presentations) - Content likely to be referenced or reused multiple times\n# Don\'t use artifacts for... - Simple, informational, or short content, such as brief code snippets, mathematical equations, or small examples - Primarily explanatory, instructional, or illustrative content, such as examples provided to clarify a concept - Suggestions, commentary, or feedback on existing artifacts - Conversational or explanatory content that

### Running locally with `transformers`

In [11]:
import torch
from transformers import pipeline
from hf_hub_prompts import download_prompt

model_id = "meta-llama/Llama-3.2-1B-Instruct"

pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

prompt_template = download_prompt(repo_id="MoritzLaurer/closed_system_prompts", filename="jokes-prompt.yaml")
messages = prompt_template.format_messages(name="Peter", assistant_attribute="speaks like a pirate")

outputs = pipe(
    messages,
    max_new_tokens=256,
)

print(outputs[0]["generated_text"][-1])


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


{'role': 'assistant', 'content': 'Yer name be Peter, eh? Alright then, matey, here be a joke fer ye:\n\nWhy did Peter the pirate take his anchor to the party?\n\n(pause for dramatic effect)\n\nBecause he wanted to "drop" some knowledge! (get it, like anchor?) Arrr, I hope that made ye laugh, Peter!'}


#### Load prompts from dataset repos

In [2]:
from hf_hub_prompts import download_prompt, list_prompts

print(list_prompts(repo_id="MoritzLaurer/dataset_prompts", repo_type="dataset"))

prompt_template = download_prompt(repo_id="MoritzLaurer/dataset_prompts", filename="fineweb-edu-prompt.yaml", repo_type="dataset")

# populate the prompt
text_to_score = "The quick brown fox jumps over the lazy dog"
messages = prompt_template.format_messages(text_to_score=text_to_score)

messages

['fineweb-edu-prompt.yaml']


[{'role': 'user',
  'content': 'Below is an extract from a web page. Evaluate whether the page has a high educational value and could be useful in an educational setting for teaching from primary school to grade school levels using the additive 5-point scoring system described below. Points are accumulated based on the satisfaction of each criterion:\n- Add 1 point if the extract provides some basic information relevant to educational topics, even if it includes some irrelevant or non-academic content like advertisements and promotional material. - Add another point if the extract addresses certain elements pertinent to education but does not align closely with educational standards. It might mix educational content with non-educational material, offering a superficial overview of potentially useful topics, or presenting information in a disorganized manner and incoherent writing style. - Award a third point if the extract is appropriate for educational use and introduces key concepts 

In [9]:
# Check which input variables the prompt template requires
print(prompt_template["input_variables"])
# ['text_to_score']

text_to_score = "The quick brown fox jumps over the lazy dog"
messages = prompt_template.format_messages(
    text_to_score=text_to_score, 
)
messages

['text_to_score']


[{'role': 'user',
  'content': 'Below is an extract from a web page. Evaluate whether the page has a high educational value and could be useful in an educational setting for teaching from primary school to grade school levels using the additive 5-point scoring system described below. Points are accumulated based on the satisfaction of each criterion:\n- Add 1 point if the extract provides some basic information relevant to educational topics, even if it includes some irrelevant or non-academic content like advertisements and promotional material. - Add another point if the extract addresses certain elements pertinent to education but does not align closely with educational standards. It might mix educational content with non-educational material, offering a superficial overview of potentially useful topics, or presenting information in a disorganized manner and incoherent writing style. - Award a third point if the extract is appropriate for educational use and introduces key concepts 

In [10]:
# test prompt with local llama
import torch
from transformers import pipeline

model_id = "meta-llama/Llama-3.2-1B-Instruct"  # prompt was original created for meta-llama/Meta-Llama-3-70B-Instruct

pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

outputs = pipe(messages, max_new_tokens=512)

print(outputs[0]["generated_text"][-1])


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


{'role': 'assistant', 'content': 'Based on the extract provided, I would evaluate the educational value of the page as follows:\n\n- The extract provides some basic information relevant to educational topics, such as the definition of the quick brown fox and its jumping ability.\n- However, the extract does not address certain elements pertinent to education, like the concept of jumping and its significance in everyday life.\n- The extract is appropriate for educational use and introduces key concepts relevant to school curricula.\n- The writing style is clear and coherent, making it suitable for a basic tutorial or introductory section of a textbook.\n- The content is coherent, focused, and valuable for structured learning.\n\nTotal score: 4 points\n\nJustification: The extract provides a basic understanding of a fundamental concept, but its execution lacks depth and relevance. The inclusion of advertisements and promotional material detracts from its educational value. While it addre

#### Open Weigth Image Models

In [10]:
from hf_hub_prompts import download_prompt

# download image prompt template
prompt_template = download_prompt(repo_id="MoritzLaurer/open_models_special_prompts", filename="internvl2-bbox-prompt.yaml")

# populate prompt
image_url = "https://unsplash.com/photos/ZVw3HmHRhv0/download?ixid=M3wxMjA3fDB8MXxhbGx8NHx8fHx8fDJ8fDE3MjQ1NjAzNjl8&force=true&w=1920"
region_to_detect = "the bird"
messages = prompt_template.format_messages(image_url=image_url, region_to_detect=region_to_detect, client="openai")

messages

[{'role': 'user',
  'content': [{'type': 'image_url',
    'image_url': {'url': 'https://unsplash.com/photos/ZVw3HmHRhv0/download?ixid=M3wxMjA3fDB8MXxhbGx8NHx8fHx8fDJ8fDE3MjQ1NjAzNjl8&force=true&w=1920'}},
   {'type': 'text',
    'text': 'Please provide the bounding box coordinate of the region this sentence describes: <ref>the bird</ref>'}]}]

In [8]:
# download image prompt template
prompt_template = download_prompt(repo_id="MoritzLaurer/open_models_special_prompts", filename="internvl2-objectdetection-prompt.yaml")

# populate prompt
image_url = "https://unsplash.com/photos/ZVw3HmHRhv0/download?ixid=M3wxMjA3fDB8MXxhbGx8NHx8fHx8fDJ8fDE3MjQ1NjAzNjl8&force=true&w=1920"
messages = prompt_template.format_messages(image_url=image_url, client="openai")

messages

[{'role': 'user',
  'content': [{'type': 'image_url',
    'image_url': {'url': 'https://unsplash.com/photos/ZVw3HmHRhv0/download?ixid=M3wxMjA3fDB8MXxhbGx8NHx8fHx8fDJ8fDE3MjQ1NjAzNjl8&force=true&w=1920'}},
   {'type': 'text',
    'text': 'Please detect and label all objects in the following image and mark their positions.'}]}]

In [9]:
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()

ENDPOINT_URL = "https://tkuaxiztuv9pl4po.us-east-1.aws.endpoints.huggingface.cloud" + "/v1/" 

# initialize the OpenAI client but point it to an endpoint running vLLM or TGI
client = OpenAI(
    base_url=ENDPOINT_URL, 
    api_key=os.getenv("HF_TOKEN")
)

response = client.chat.completions.create(
    model="/repository", # with vLLM deployed on HF endpoint, this needs to be /repository since there are the model artifacts stored
    messages=messages,
)

response.choices[0].message.content



'Sure, I will detect and label all objects in the image and mark their positions.\n\n```\neuropean robin[[0, 406, 515, 950]]\n```'

### Test other functions

In [4]:
from hf_hub_prompts import download_prompt, list_prompts, PromptTemplate

list_prompts(repo_id="MoritzLaurer/closed_system_prompts")

['claude-3-5-artifacts-leak-210624.yaml',
 'claude-3-5-sonnet-text-090924.yaml',
 'claude-3-5-sonnet-text-image-090924.yaml',
 'jokes-prompt.yaml',
 'openai-metaprompt-audio.yaml',
 'openai-metaprompt-text.yaml']

### Upload prompts to Hub
TODO

In [ ]:


from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="prompts/internvl-bbox-prompt.yaml",
    path_in_repo="prompts/internvl-bbox-prompt.yaml",
    repo_id="MoritzLaurer/model_specific_prompts",
    repo_type="model",
)